# Import

In [4]:
import os, math, time, random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
import glob
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import random_split, Dataset, DataLoader
from torchvision import transforms
import torch, torchvision
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import datasets, transforms, models
from torch.cuda.amp import autocast, GradScaler
from tqdm.notebook import tqdm
import random


In [5]:
torch.manual_seed(42); np.random.seed(42); random.seed(42)
torch.backends.cudnn.benchmark = True

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Data

In [16]:
!wget https://hydranets-data.s3.eu-west-3.amazonaws.com/UTKFace.zip

--2025-08-27 08:03:27--  https://hydranets-data.s3.eu-west-3.amazonaws.com/UTKFace.zip
Resolving hydranets-data.s3.eu-west-3.amazonaws.com (hydranets-data.s3.eu-west-3.amazonaws.com)... 16.12.20.42, 3.5.204.146
Connecting to hydranets-data.s3.eu-west-3.amazonaws.com (hydranets-data.s3.eu-west-3.amazonaws.com)|16.12.20.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131852014 (126M) [application/zip]
Saving to: ‘UTKFace.zip’

UTKFace.zip         100%[===================>] 125.74M  26.5MB/s    in 5.5s    

2025-08-27 08:03:33 (23.0 MB/s) - ‘UTKFace.zip’ saved [131852014/131852014]



In [17]:
!jar xf UTKFace.zip

In [69]:
image_paths = sorted(glob.glob("UTKFace/*.jpg.chip.jpg"))
print(image_paths[0])   # first part of the name is the age

UTKFace/100_0_0_20170112213500903.jpg.chip.jpg


In [19]:
images = []
ages = []

for path in image_paths:
    filename = path[8:].split("_")
    if len(filename)==4:
        images.append(np.array(Image.open(path)))
        ages.append(int(filename[0]))

if len(images)==len(ages):
    print(len(images))

23704


In [20]:
np.max(ages)

116

In [21]:
min_age, max_age = min(ages), max(ages)
ages_normalized = np.array(ages)/max_age
min_age_normalized, max_age_normalized = min(ages_normalized), max(ages_normalized)
print('max age:', max_age)
print('min age:', min_age)
print('max age normalized:', max_age_normalized)
print('min age normalized:', min_age_normalized)


max age: 116
min age: 1
max age normalized: 1.0
min age normalized: 0.008620689655172414


In [36]:
IMG_SIZE = 224
MAX_AGE = 116

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.9, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.5)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class UTKFace(Dataset):
    def __init__(self, image_paths, transform):

        # Define the Transforms
        self.transform = transform

        # Set Inputs and Labels
        self.image_paths = image_paths
        self.images = []
        self.ages = []
        self.ages_normalized = []

        for path in image_paths:
            filename = path[8:].split("_")
            if len(filename)==4:
                if(int(filename[0]) > 60): continue
                self.images.append(path)
                self.ages.append(int(filename[0]))

        # self.ages_normalized = np.array(self.ages) / MAX_AGE

    def __len__(self):
         return len(self.images)

    def __getitem__(self, index):
        # Load an Image
        img = Image.open(self.images[index]).convert('RGB')
        # Transform it
        img = self.transform(img)

        # Get the Labels
        age = torch.tensor(self.ages[index], dtype=torch.float32)   # / ages_normalized

        # Return the sample of the dataset
        sample = img, age
        return sample

In [38]:
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.2

train_num = int(TRAIN_SPLIT * len(image_paths))
val_num   = len(image_paths) - train_num

print('Number of train samples', train_num)
print('Number of validation Samples', val_num)

Number of train samples 18965
Number of validation Samples 4742


In [39]:
(train_dataset, valid_dataset) = random_split(image_paths,[train_num, val_num],
                                              generator=torch.Generator().manual_seed(42))

In [40]:
BATCH_SIZE = 64

train_dataloader = DataLoader(UTKFace(train_dataset, train_transform), shuffle=True, batch_size=BATCH_SIZE,
                              num_workers=2, pin_memory=True, persistent_workers=True)
val_dataloader = DataLoader(UTKFace(valid_dataset, val_transform), shuffle=False, batch_size=BATCH_SIZE,
                            num_workers=2, pin_memory=True, persistent_workers=True)


In [41]:
imgs, ages = next(iter(train_dataloader))
print("Batch shape:", imgs.shape)  # [64, 3, 224, 224]
print("Ages:", ages[:10])          # first 10 ages


Batch shape: torch.Size([64, 3, 224, 224])
Ages: tensor([26., 35., 28., 56., 26., 52., 40., 26., 27., 26.])


# My ResNet Model Implementation from Scratch

In [43]:
class BottleNeck(nn.Module):
    """
    Inputs:
        Number of input channel for this BottleNeck block
        Number of output channel from first Conv2d layer in this BottleNeck block
    """
    expansion=4

    def __init__(self, in_ch, out_ch, stride=1):
        super(BottleNeck, self).__init__()

        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)     # padding = same = (f-1)/2
        self.bn2 = nn.BatchNorm2d(out_ch)
        self.conv3 = nn.Conv2d(out_ch, self.expansion*out_ch, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*out_ch)
        self.relu = nn.ReLU()

        self.need_proj = (stride!=1) or (in_ch!=out_ch*self.expansion)      # need downsample and/or expand
        # stride == 2 -> first bottle neck block in any layer except layer 1 (that have 3 blocks) -> expand channels + downsample H,W
        # in_ch == out_ch in first bottle neck block in layer 1 (that have 3 blocks) and its stride is 1 -> expand only

        if self.need_proj:
            self.proj = nn.Sequential(
                nn.Conv2d(in_ch, self.expansion*out_ch, kernel_size=1, stride=stride, padding=0, bias=False),
                nn.BatchNorm2d(self.expansion*out_ch)
            )
        else:
            self.proj = nn.Identity()

        nn.init.constant_(self.bn3.weight, 0.0)     # start with res branch -> off, they found it is better

    def forward(self, x):
        identity = self.proj(x)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        x += identity
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels=3):     # layers -> [3, 4, 6, 3] in resnet 50
        super(ResNet, self).__init__()

        self.block = block

        self.conv = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)


        self.layer1 = self._make_layer(64, layers[0], 1)               # no stride in first block (no reduction in W,H)
        self.layer2 = self._make_layer(128, layers[1], 2)              # there is Stride = 2 in first bottleneck in each layer
        self.layer3 = self._make_layer(256, layers[2], 2)
        self.layer4 = self._make_layer(512, layers[3], 2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))     # 1x1xChannels
        self.fc = nn.Linear(2048, 1)

    def _make_layer(self, first_conv_channels, number_of_blocks, first_block_stride):

        if first_block_stride==1:
            layers = [self.block(first_conv_channels, first_conv_channels, first_block_stride)]
        else:
            layers = [self.block(first_conv_channels*2, first_conv_channels, first_block_stride)]

        for i in range(number_of_blocks-1):
            layers.append(self.block(first_conv_channels*self.block.expansion, first_conv_channels, 1))

        return nn.Sequential(*layers)

    def forward(self, x):   # to make each of the four layers each of [3, 4, 6, 3] BottleNeck blocks
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        # x = torch.sigmoid(x)
        return x


def ResNet50(img_channel=3):
    return ResNet(BottleNeck, [3, 4, 6, 3], img_channel)


In [32]:
def test():
    BATCH_SIZE = 4
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = ResNet50(img_channel=3).to(device)
    total = sum(p.numel() for p in net.parameters())
    trainable = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print(f"Model params: {total:,}")

    x = torch.randn(BATCH_SIZE, 3, 224, 224, device=device)
    y = net(x)
    print(y.shape)  # should be [BATCH_SIZE, 1]

test()

Model params: 23,510,081
torch.Size([4, 1])


# Model Training

In [44]:
model = ResNet50().to(device)
criterion = nn.SmoothL1Loss(beta=6) #nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0001)
#            torch.optim.AdamW(model.parameters(), lr=3e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=3)
num_epochs = 50

In [45]:
MAX_AGE = 1 # no norm

def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, train_mae = 0.0, 0.0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for imgs, ages in pbar:
            imgs, ages = imgs.to(device), ages.to(device)
            optimizer.zero_grad()
            preds = model(imgs).squeeze(1)
            loss = criterion(preds, ages)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()

            train_loss += loss.item() * imgs.size(0)
            train_mae  += (preds*MAX_AGE - ages*MAX_AGE).abs().sum().item()

            pbar.set_postfix(loss=f"{loss.item():.4f}")

        train_loss /= len(train_loader.dataset)
        train_mae  /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss, val_mae = 0.0, 0.0
        with torch.no_grad():
          for imgs, ages in val_loader:
              imgs, ages = imgs.to(device), ages.to(device)
              preds = model(imgs).squeeze(1)        # in [0,1]
              loss = criterion(preds, ages)
              val_loss += loss.item() * imgs.size(0)
              val_mae  += (preds*MAX_AGE - ages*MAX_AGE).abs().sum().item()

        val_loss /= len(val_loader.dataset)
        val_mae  /= len(val_loader.dataset)

        scheduler.step(val_mae)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f} | Train MAE: {train_mae:.2f} yrs | "
              f"Val Loss: {val_loss:.4f} | Val MAE: {val_mae:.2f} yrs")


In [46]:
train_model(model, train_dataloader, val_dataloader, epochs=num_epochs)

torch.save(model.state_dict(), "resnet50_age.pth")


Epoch 1/50:   0%|          | 0/267 [00:00<?, ?it/s]

Epoch 1/50 | Train Loss: 9.0904 | Train MAE: 11.69 yrs | Val Loss: 8.4756 | Val MAE: 11.06 yrs


Epoch 2/50:   0%|          | 0/267 [00:00<?, ?it/s]

Epoch 2/50 | Train Loss: 8.0593 | Train MAE: 10.65 yrs | Val Loss: 9.5223 | Val MAE: 12.17 yrs


Epoch 3/50:   0%|          | 0/267 [00:00<?, ?it/s]

...

Epoch 48/50 | Train Loss: 1.9826 | Train MAE: 3.94 yrs | Val Loss: 2.6692 | Val MAE: 4.74 yrs


Epoch 49/50:   0%|          | 0/267 [00:00<?, ?it/s]

Epoch 49/50 | Train Loss: 1.9938 | Train MAE: 3.95 yrs | Val Loss: 2.6725 | Val MAE: 4.76 yrs


Epoch 50/50:   0%|          | 0/267 [00:00<?, ?it/s]

Epoch 50/50 | Train Loss: 2.0321 | Train MAE: 3.99 yrs | Val Loss: 2.6764 | Val MAE: 4.76 yrs


In [68]:
# saving model
torch.save(model.state_dict(), "/kaggle/working/resnet50_age_best.pth")

In [70]:
model.eval()
with torch.no_grad():
    imgs, ages_norm = next(iter(val_dataloader))      
    # img = imgs[0].unsqueeze(0).to(device)  
    i=0
    for img in imgs[:65]:
        img = img.unsqueeze(0).to(device)
        true_age = (ages_norm[i].item() * MAX_AGE); i+=1;          
        pred_norm = model(img).squeeze(1).item()  
        pred_age = pred_norm * MAX_AGE
        print(f"True Age: {true_age:.0f}, Predicted Age: {pred_age:.1f}")


True Age: 57, Predicted Age: 45.2
True Age: 9, Predicted Age: 10.0
True Age: 27, Predicted Age: 38.6
True Age: 23, Predicted Age: 21.0
True Age: 25, Predicted Age: 30.6
True Age: 2, Predicted Age: 0.6
True Age: 46, Predicted Age: 35.1
True Age: 30, Predicted Age: 29.6
True Age: 24, Predicted Age: 24.8
True Age: 51, Predicted Age: 37.7
True Age: 10, Predicted Age: 21.1
True Age: 1, Predicted Age: 2.3
True Age: 18, Predicted Age: 24.2
True Age: 57, Predicted Age: 52.6
True Age: 46, Predicted Age: 43.1
True Age: 35, Predicted Age: 43.6
True Age: 46, Predicted Age: 45.0
True Age: 40, Predicted Age: 26.3
True Age: 1, Predicted Age: 0.4
True Age: 27, Predicted Age: 36.0
True Age: 56, Predicted Age: 51.0
True Age: 4, Predicted Age: 0.6
True Age: 36, Predicted Age: 38.6
True Age: 1, Predicted Age: 2.0
True Age: 48, Predicted Age: 42.9
True Age: 13, Predicted Age: 20.0
True Age: 26, Predicted Age: 23.8
True Age: 3, Predicted Age: 1.9
True Age: 26, Predicted Age: 25.7
True Age: 1, Predicted Age: